<a href="https://colab.research.google.com/github/Hamza-Shahzad/CIFAR-100-Model/blob/master/pretrained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# from hyperopt import hp
# from ray.tune.suggest.hyperopt import HyperOptSearch
import torch
from torch.autograd import Variable
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.models as models
import torchvision.transforms as transforms
import torch.optim as optim

dir(models)

['AlexNet',
 'DenseNet',
 'GoogLeNet',
 'Inception3',
 'MNASNet',
 'MobileNetV2',
 'ResNet',
 'ShuffleNetV2',
 'SqueezeNet',
 'VGG',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_utils',
 'alexnet',
 'densenet',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'detection',
 'googlenet',
 'inception',
 'inception_v3',
 'mnasnet',
 'mnasnet0_5',
 'mnasnet0_75',
 'mnasnet1_0',
 'mnasnet1_3',
 'mobilenet',
 'mobilenet_v2',
 'resnet',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnext101_32x8d',
 'resnext50_32x4d',
 'segmentation',
 'shufflenet_v2_x0_5',
 'shufflenet_v2_x1_0',
 'shufflenet_v2_x1_5',
 'shufflenet_v2_x2_0',
 'shufflenetv2',
 'squeezenet',
 'squeezenet1_0',
 'squeezenet1_1',
 'utils',
 'vgg',
 'vgg11',
 'vgg11_bn',
 'vgg13',
 'vgg13_bn',
 'vgg16',
 'vgg16_bn',
 'vgg19',
 'vgg19_bn',
 'video',
 'wide_resnet101_2',
 'wide_resnet50_2']

In [0]:
batch_size = 50
lr = 1e-3
num_classes = 100
num_epochs = 20

In [0]:
net = models.resnet152(pretrained=True)
print(net)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/checkpoints/resnet152-b121ed2d.pth
100%|██████████| 230M/230M [00:02<00:00, 106MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [0]:
from torchvision import transforms

transform = transforms.Compose([
                                transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(
                                    mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225]
                                )
])

In [0]:
train_data = dsets.CIFAR100(root = './data', train = True,
                            transform = transform, download = True)

test_data = dsets.CIFAR100(root = './data', train = False,
                           transform = transform)

0it [00:00, ?it/s]

 99%|█████████▉| 167534592/169001437 [00:11<00:00, 17040324.26it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data


In [0]:
train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                        batch_size = batch_size,
                                        shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                       batch_size = batch_size, 
                                       shuffle = False)

In [0]:
net = models.resnet152(pretrained=True)
# Freeze all the preexisting layers
for param in net.parameters():
    param.requires_grad = False
# Replace the last layer with one of output size = 100
net.fc = nn.Linear(in_features=2048, out_features=num_classes, bias=True)

In [0]:
if torch.cuda.is_available():
  net.cuda()

169009152it [00:29, 17040324.26it/s]                               

In [0]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.fc.parameters(), lr=lr)

In [0]:
print("Configuration:",
      "batch_size", batch_size,
      "lr", lr,
      "num_classes", num_classes,
      "num_epochs", num_epochs
      )

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_gen):
      images = Variable(images).cuda()
      labels = Variable(labels).cuda()
    
      optimizer.zero_grad()
      outputs = net(images)
      loss = loss_function(outputs, labels)
      loss.backward()
      optimizer.step()
    
      if (i+1) % 100 == 0:
          print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.item()))

Configuration: batch_size 50 lr 0.001 num_classes 100 num_epochs 20
Epoch [1/20], Step [100/1000], Loss: 2.9781
Epoch [1/20], Step [200/1000], Loss: 2.1661
Epoch [1/20], Step [300/1000], Loss: 1.7098
Epoch [1/20], Step [400/1000], Loss: 2.0722
Epoch [1/20], Step [500/1000], Loss: 1.6844
Epoch [1/20], Step [600/1000], Loss: 1.8391
Epoch [1/20], Step [700/1000], Loss: 1.8570
Epoch [1/20], Step [800/1000], Loss: 1.6361
Epoch [1/20], Step [900/1000], Loss: 1.5777
Epoch [1/20], Step [1000/1000], Loss: 1.5671
Epoch [2/20], Step [100/1000], Loss: 0.8363
Epoch [2/20], Step [200/1000], Loss: 1.3346
Epoch [2/20], Step [300/1000], Loss: 1.5131
Epoch [2/20], Step [400/1000], Loss: 1.6810
Epoch [2/20], Step [500/1000], Loss: 1.3887
Epoch [2/20], Step [600/1000], Loss: 1.6898
Epoch [2/20], Step [700/1000], Loss: 1.6525
Epoch [2/20], Step [800/1000], Loss: 1.4094
Epoch [2/20], Step [900/1000], Loss: 1.3979
Epoch [2/20], Step [1000/1000], Loss: 1.3797
Epoch [3/20], Step [100/1000], Loss: 1.5833
Epoch 

In [0]:
#@title Evaluating the accuracy of the model

correct = 0
total = 0
for images,labels in test_gen:
  images = Variable(images).cuda()
  labels = labels.cuda()
  
  output = net(images)
  _, predicted = torch.max(output, 1)
  correct += (predicted == labels).sum()
  total += labels.size(0)

print('Accuracy of the model: %.3f %%' %((100*correct)/(total+1)))

Accuracy of the model: 60.000 %


https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz